In [3]:
import pandas as pd
import json
from sbscorer.sbdata.FlipsideApi import FlipsideApi
import os
import numpy as np
from pathlib import Path

In [4]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)

# Initialize Flipside API
api_key = os.environ['FLIPSIDE_API_KEY']
flipside_api = FlipsideApi(api_key, max_address=100)

In [5]:
PATH_TO_EXPORT 

'c:\\Users\\DELL Admin\\Documents\\ODC\\RegenRangers\\tx_data\\gr18'

In [34]:
sql_template = """
 with dai as (
SELECT
  tx_hash,
  block_timestamp,
  origin_from_address as voter,
  'DAI' as token,
  amount_usd
from
  optimism.core.ez_token_transfers
where
  contract_address = '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1'
and origin_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
and block_timestamp between '2023-08-15 12:00:00.000' and '2023-08-29 12:00:00.000'

), eth AS
(
SELECT
  tx_hash,
  block_timestamp,
  origin_from_address as voter,
  'ETH' as token,
  amount_usd
from
  optimism.core.ez_eth_transfers
where
  origin_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
and eth_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
and block_timestamp between '2023-08-15 12:00:00.000' and '2023-08-29 12:00:00.000'
),

all_votes as (
SELECT * from eth
 union all 
select * FROM dai
),

unique_voter as (SELECT DISTINCT voter from all_votes)

SELECT 
tx.TX_HASH,
tx.BLOCK_TIMESTAMP,
tx.FROM_ADDRESS,
tx.TO_ADDRESS,
tx.GAS_LIMIT,
tx.GAS_USED,
tx.TX_FEE,
tx.ETH_VALUE
FROM optimism.core.fact_transactions tx
INNER JOIN
unique_voter v ON tx.FROM_ADDRESS = v.voter OR tx.TO_ADDRESS = v.voter
WHERE tx.BLOCK_TIMESTAMP 
between CURRENT_DATE - interval '%d day'
AND CURRENT_DATE - interval '%d day'
AND tx.STATUS = 'SUCCESS'  
"""

In [12]:
query_result_set = flipside_api.sdk.query(sql,
                                              max_age_minutes=flipside_api.MAX_AGE_MINUTES,
                                              page_size=50000,
                                              page_number=1,
                                              timeout_minutes=flipside_api.TIMEOUT_MINUTES,
                                              ttl_minutes=flipside_api.TTL_MINUTES,
                                              cached=flipside_api.CACHED,
                                              retry_interval_seconds=flipside_api.RETRY_INTERVAL_SECONDS)

In [20]:
def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return f"{size:.{decimal_places}f} {unit}"
        size /= 1024.0
    return f"{size:.{decimal_places}f} PB"

print(human_readable_size(query_result_set.run_stats.bytes))


239.68 MB


In [63]:
from datetime import datetime

# Get today's date
today = datetime.today().date()

# Date for comparison
comparison_date = datetime(2021, 11, 11).date()

# Calculate the difference in days
days_difference = (today - comparison_date).days

print(days_difference)


657


In [36]:
n_days = 100
PAGE_SIZE = 50000

flipside_api = FlipsideApi(api_key, page_size=PAGE_SIZE)
ls_df = []
for i in range (0, 8):
    sql = sql_template % ((i+1)*n_days, i*n_days) # between oldest date and current date
    ls_df.append(flipside_api.execute_query(sql))


 with dai as (
SELECT
  tx_hash,
  block_timestamp,
  origin_from_address as voter,
  'DAI' as token,
  amount_usd
from
  optimism.core.ez_token_transfers
where
  contract_address = '0xda10009cbd5d07dd0cecc66161fc93d7c9000da1'
and origin_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
and block_timestamp between '2023-08-15 12:00:00.000' and '2023-08-29 12:00:00.000'

), eth AS
(
SELECT
  tx_hash,
  block_timestamp,
  origin_from_address as voter,
  'ETH' as token,
  amount_usd
from
  optimism.core.ez_eth_transfers
where
  origin_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
and eth_to_address = '0x15fa08599eb017f89c1712d0fe76138899fdb9db'
and block_timestamp between '2023-08-15 12:00:00.000' and '2023-08-29 12:00:00.000'
),

all_votes as (
SELECT * from eth
 union all 
select * FROM dai
),

unique_voter as (SELECT DISTINCT voter from all_votes)

SELECT 
tx.TX_HASH,
tx.BLOCK_TIMESTAMP,
tx.FROM_ADDRESS,
tx.TO_ADDRESS,
tx.GAS_LIMIT,
tx.GAS_USED,
tx.TX_FEE,
tx.ETH_V

In [38]:
df = pd.concat(ls_df)

In [42]:
os.path.join(PATH_TO_EXPORT, 'optimism.csv')

'c:\\Users\\DELL Admin\\Documents\\ODC\\RegenRangers\\tx_data\\gr18\\optimism.csv'

In [45]:
if not os.path.exists(PATH_TO_EXPORT):
    os.makedirs(PATH_TO_EXPORT)

df.to_csv(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.csv'), index=False)

In [47]:
df.to_pickle(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.pkl'))

In [48]:
PATH_TO_EXPORT

'c:\\Users\\DELL Admin\\Documents\\ODC\\RegenRangers\\tx_data\\gr18'

In [50]:
tx_hash = '0x2891204ef83c263cf8a8c9125d339b4e2568d29da548a71c21da24412ad389de'

df_loaded = pd.read_pickle(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.pkl'))

In [53]:
tx_hash in df_loaded['tx_hash'].values

False

In [56]:
ls_df_missing = []
sql = sql_template % (250, 200) # between oldest date and current date
ls_df_missing.append(flipside_api.execute_query(sql))

sql = sql_template % (300, 250) # between oldest date and current date
ls_df_missing.append(flipside_api.execute_query(sql))

df_missing = pd.concat(ls_df_missing)

In [57]:
new_df = pd.concat([df_loaded, df_missing])

In [58]:
tx_hash in new_df['tx_hash'].values

True

In [60]:
df_not_dupplicate = new_df[~new_df.duplicated()]
df_not_dupplicate

,tx_hash,block_timestamp,from_address,to_address,gas_limit,gas_used,tx_fee,eth_value,__row_index
0,0x863c5b543d29b329891fdec341845a73c2525e6dc212...,2023-08-24T20:22:03.000Z,0xf4af0314a06ea7376da5bdecc33f6f9800f1ae28,0x7f5c764cbc14f9669b88837ca1490cca17c31607,53067,53067,0.000029,0.000000,0
1,0xfdffb6eb77e72ca9f54297ebcc7644ea55db521f64c7...,2023-08-24T22:05:41.000Z,0x87dbffd0ed90b2252856faf753437a7bc9fcfa7b,0xb009b5707e6e0dc9830a1262de37f055972e6480,219895,139765,0.000058,0.000777,1
2,0x7b416f625fd29c2da17a7ad35b1c6dc7e0cf5b548249...,2023-08-24T16:10:17.000Z,0x532c1fc2940ec38d5c5e7fc8b8aa4aaddcaa17d3,0x46ce3b08bdef4461fb444d3746fd97f0af11dccf,731592,467423,0.000093,0.000000,2
3,0xbed4ce09f5f5c4d7aac7e310dfce11d94bf86f7da6d5...,2023-08-24T16:09:33.000Z,0x0596c217a79ea00e5dafc43778138ea61fd67ee2,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119625,119625,0.000087,0.000000,3
4,0xb60502458c8de31606b84e0aed102ac83629726d8734...,2023-08-24T22:08:15.000Z,0xc1991c8bda29507991eeb230ff2063c2ea74a34c,0xa062ae8a9c5e11aaa026fc2670b0d65ccc8b2858,246899,229592,0.000070,0.000000,4
...,...,...,...,...,...,...,...,...,...
22813,0x4af84bcaf2c34b3a820c2f23bdc1b3d4fab8a03bec53...,2022-12-12T12:57:26.000Z,0xb0f9290abacddb2fb3e467752d926a91440b7dde,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,91141,69706,0.000071,0.000000,1072813
22814,0x029d7e2301326680f85a9cb58181881c4821db79c021...,2022-12-12T00:13:04.000Z,0x868490398931ffead3c38d6d9af71f4f06d3f4f6,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,158088,122918,0.000146,0.000000,1072814
22815,0xa0621fe1e8e0475eba974cb4d6896693be26f99e28b7...,2022-12-12T14:16:27.000Z,0xa8d8802fcdbfa82fe71657f28091344fca0dad75,0x062016cd29fabb26c52bab646878987fc9b0bc55,170577,170577,0.000079,0.000000,1072815
22816,0xf9368f08adc8b95eee066446b4681fb02ec54ae6ba07...,2022-12-12T14:11:41.000Z,0x70e983a8303f73cd7af87e91d528b7074dfade6e,0x2f05e799c61b600c65238a9df060caba63db8e78,187713,178957,0.000129,0.000772,1072816


In [61]:
if not os.path.exists(PATH_TO_EXPORT):
    os.makedirs(PATH_TO_EXPORT)

duplicate_rows.to_csv(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.csv'), index=False)
duplicate_rows.to_pickle(os.path.join(PATH_TO_EXPORT, 'gr_18_optimism_all_tx.pkl'))

In [62]:
PATH_TO_EXPORT

'c:\\Users\\DELL Admin\\Documents\\ODC\\RegenRangers\\tx_data\\gr18'